##  Study on how to give initial data. 

We want to have a very isotropic data with a given frequency, or perhaps a distribution (thermal?) for $phi$ and its time derivative. Here
We do in 2dim.

We first create a distribution of values on Fourier space both for $ϕ $ and its time derivative, $\phi_t$.
I think we need a thermal distribution, that is we fix a temperature and put particles at that temperature.
That is we shall use the $\omega = |k|$ rule, we take positive so as to have particles and not antiparticles.



In [1]:
using Plots
using Distributions
using FFTW
using LinearAlgebra
using Revise

In [ ]:
function get_index_p(x,J,L)
    floor(Int64,(x>=0) ? mod(x / L[1] * J[1] + 1 ,J[1]) : -mod(-x / L[1] * J[1] ,J[1])+J+1 )
    end
    
function get_index_p(x::Vector,J,L)
        j = Vector{Int64}(undef,length(J))
        for i in 1:length(J)
            j[i] = get_index_p(x[i],J[i],L[i])
        end
        return j[:]
end

function n(ϕ,ϕ_t,Box_x,J)
        V = volume(Box_x)
        dx = differentials(Box_x,J)
        n = real(im*sum(ϕ.*conj.(ϕ_t).-conj.(ϕ).*ϕ_t))/2#*prod(dx)/2)
        return n, n/prod(J)
end

We first look at the scheme for a distribution which resembles a delta function $k$ in Fourier space.
It gives a rather homogeneous distribution is space.

In [227]:
#J = (2001,2001)
J = (1001,1001)
Box_x = [0.0, 500, 0.0, 500]
dx = differentials(Box_x,J)
κ = 2π./(Box_x[2:2:end] - Box_x[1:2:end-1])
V = volume(Box_x)
ϕ = Array{ComplexF64}(undef,J...)
ϕ_t = copy(ϕ)
g = copy(ϕ)
g_t = copy(ϕ)
g_p = copy(ϕ)

m = 1
m2 = m^2

N = 10000 #number of particles
 
function f(k,k0,δ)
    #(kx,ky) = k
    if abs(k'*k - k0^2) < δ^2
        return (1.0 + 0.0im)/(2π*(k0*2*δ)) # normalized so that the integral is 1
    else
        return 0.0 + 0.0im
    end
end

function h(k,k0)
    if abs(k'*k)< k0^2
        return (1.0 + 0.0im)/2*π/k0^2
    else
        return 0.0
    end
end

# k0= 500; δ = 1000 # para J=2001
k0 = 1; δ = 0.5

kx = fftfreq(J[1]).*κ[1]*J[1]
ky = fftfreq(J[2]).*κ[2]*J[2]
kx_p = sort(kx)
ky_p = sort(ky)
ω(kx,ky,m2) = sqrt((kx^2+ky^2+m2))

for i in 1:J[1]
    for j in 1:J[2]
        ω_val = ω(kx[i],ky[j],m2)
        g[i,j] = exp(im*2*π*rand())*f([kx[i],ky[j]],k0,δ)/sqrt(2.0*ω_val)
        g_t[i,j] = im*sqrt(2.0*ω_val)*g[i,j]
        ω_val_p = ω(kx_p[i],ky_p[j],m2)
        g_p[i,j] = f([kx_p[i],ky_p[j]],k0,δ)*exp(im*2*π*rand())/sqrt(2.0*ω_val_p)
    end
end



In [ ]:
if false
    g = fill(0.0+im*0.0,J...)
    g_t = copy(g)

    g[5,5] = exp(im*2*π*rand())
    g_t[5,5] = im*(sqrt(kx[5]^2+ky[5]^2 + m2))*g[5,5]
end

In [ ]:

sum(abs.(g))*prod(κ)
#heatmap(ky_p,kx_p,real.(g_p[:,:]), aspectratio = 1)
#surface(ky_p,kx_p,real.(g[:,:]), aspectratio = 1)


In [229]:
#ifft(g,J...)
p_i = plan_ifft(g; flags=FFTW.ESTIMATE, timelimit=Inf);

In [230]:
ϕ = p_i*g*sqrt(prod(J));
ϕ_t = p_i*g_t*sqrt(prod(J));


In [ ]:

sum(ϕ.*conj.(ϕ))/prod(J)

In [ ]:

@show norm(ϕ)
@show mean(ϕ)
@show var(ϕ);
@show n(ϕ,ϕ_t,Box_x,J);

In [ ]:
#heatmap(real.(ϕ), aspectratio=1)
#heatmap(imag.(ϕ), aspectratio=1)
surface(real.(ϕ), aspectratio=1)

## Thermal distribution

We use a relativistic thermal distribution. I guess we are setting the mass to 1 but I don't know the temperature.



In [233]:
includet("../PIC/PIC-1D/aux_functions/inidat_aux_functions.jl")
includet("../PIC/PIC-1D/aux_functions/aux_functions.jl")

In [ ]:
θ = 1.0
m = 1.0
m2 = m^2
norm_rel = 2.0
p_max = 10*θ
Box_p =(-p_max,p_max,-p_max,p_max)
#dp = differentials(Box_p,J)
L = zeros(J...)
L = Box_p[2:2:end] .- Box_p[1:2:end]
f_p = f_p_rel 
f_p_max = f_p_rel_max
par_f_p_1 = (θ,norm_rel,2)
#norm_rel = int_mid_point_f(f_p, par_f_p_1, [20_000,20_000], Box_p)
par_f_p = (θ, norm_rel,2)
interval_p = Box_p


In [ ]:
g_th = fill(0.0+im*0.0,J...)
gt_th = fill(0.0+im*0.0,J...)

N=10000
plt = Plots.scatter(labels=false)
for i in 1:N÷2
    kr = random_sampling_from_distribution_D(f_p,f_p_max,par_f_p,interval_p)
    j = get_index_p(kr,J,L)
    jm = get_index_p(-kr,J,L)
    if (abs(j[1]) > J[1] || abs(jm[1])) > J[1] || (abs(j[2]) > J[2] || abs(jm[2])) > J[2]
        println("j = $j, jm = $jm, kr = $kr")
    end
    #Plots.scatter!(plt,[j[1],jm[1]],[j[2],jm[2]],label=false)
    factor = 1.0/sqrt(2*ω(kr[1],kr[2],m2))
    phase_p = exp(im*rand()*2*π)
    phase_m = exp(im*rand()*2*π)
    g_th[j...] += phase_p*factor
    g_th[jm...] += phase_m*factor
    gt_th[j...] += im*phase_p/factor
    gt_th[jm...] += im*phase_m/factor
end

#heatmap(real.(g_th))
surface(real.(g_th))
#plt

In [ ]:
#heatmap(real.(g_th))
surface(abs.(imag.(g_th)).- abs.(real.(g_th)))

In [245]:
ϕ_th = p_i*g_th*sqrt(prod(J));
ϕt_th = p_i*gt_th*sqrt(prod(J));

In [ ]:
#heatmap(imag.(ϕ_th))
surface(imag.(ϕ_th))

In [ ]:
#@show sum(ϕ.*conj.(ϕ))*prod(dx)/V

n(ϕ_th,ϕt_th,Box_x,J)

In [ ]:
p=0.0
for i in 1:J[1]
    for j in 1:J[2]
        p +=